# How does Julia's performance compare to C and Python?

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [1]:
x = rand(10^7);

In [2]:
sum(x)

5.000729775489703e6

In [3]:
d = Dict() # to store the measurement results

Dict{Any,Any}()

# Python

In [4]:
using BenchmarkTools
using PyCall

## numpy

In [5]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/Users/crstnbr/opt/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>

In [6]:
numpy_sum = np.sum

PyObject <function sum at 0x7f86e90f6560>

In [7]:
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     4.012 ms (0.00% GC)
  median time:      5.919 ms (0.00% GC)
  mean time:        6.311 ms (0.00% GC)
  maximum time:     12.770 ms (0.00% GC)
  --------------
  samples:          792
  evals/sample:     1

In [8]:
d["Python (numpy)"] = minimum(b.times) / 1e6

4.0121

## hand-written

In [9]:
py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [10]:
mysum_py = py"mysum"

PyObject <function mysum at 0x7f86e8de1d40>

In [11]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):
xpy_list = PyCall.array2py(x);

In [12]:
b = @benchmark $mysum_py($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     274.249 ms (0.00% GC)
  median time:      283.410 ms (0.00% GC)
  mean time:        286.046 ms (0.00% GC)
  maximum time:     300.360 ms (0.00% GC)
  --------------
  samples:          18
  evals/sample:     1

In [13]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

274.249437

## built-in

In [14]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [15]:
b = @benchmark $pysum($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     62.559 ms (0.00% GC)
  median time:      70.323 ms (0.00% GC)
  mean time:        69.844 ms (0.00% GC)
  maximum time:     83.418 ms (0.00% GC)
  --------------
  samples:          72
  evals/sample:     1

In [16]:
d["Python (built-in)"] = minimum(b.times) / 1e6

62.559485

# C

## hand-written

In [17]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [18]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
using Libdl

In [19]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

In [20]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [21]:
c_sum(x) ≈ sum(x)

true

In [22]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.834 ms (0.00% GC)
  median time:      14.858 ms (0.00% GC)
  mean time:        14.946 ms (0.00% GC)
  maximum time:     18.442 ms (0.00% GC)
  --------------
  samples:          335
  evals/sample:     1

In [23]:
d["C"] = minimum(b.times) / 1e6

12.834408

## hand-written (with `-fast-math`)

In [24]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum_fastmath (generic function with 1 method)

In [25]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.486 ms (0.00% GC)
  median time:      5.409 ms (0.00% GC)
  mean time:        5.748 ms (0.00% GC)
  maximum time:     10.846 ms (0.00% GC)
  --------------
  samples:          870
  evals/sample:     1

In [26]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.485973

# Julia

## built-in

In [27]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.476 ms (0.00% GC)
  median time:      5.605 ms (0.00% GC)
  mean time:        5.913 ms (0.00% GC)
  maximum time:     13.741 ms (0.00% GC)
  --------------
  samples:          846
  evals/sample:     1

In [28]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

4.475965

## built-in (with `Vector{Any}`)

In [29]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 
  memory estimate:  152.59 MiB
  allocs estimate:  9999999
  --------------
  minimum time:     268.393 ms (0.00% GC)
  median time:      293.707 ms (8.73% GC)
  mean time:        293.184 ms (7.89% GC)
  maximum time:     317.714 ms (8.14% GC)
  --------------
  samples:          18
  evals/sample:     1

In [30]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

268.392747

## hand-written

In [31]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [32]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.327 ms (0.00% GC)
  median time:      14.751 ms (0.00% GC)
  mean time:        15.078 ms (0.00% GC)
  maximum time:     19.351 ms (0.00% GC)
  --------------
  samples:          332
  evals/sample:     1

In [33]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

13.327243

## hand-written (with `@simd`)

In [34]:
function mysum_simd(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @simd for a in A
        s += a
    end
    return s
end

mysum_simd (generic function with 1 method)

In [35]:
b = @benchmark mysum_simd($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.616 ms (0.00% GC)
  median time:      5.576 ms (0.00% GC)
  mean time:        5.794 ms (0.00% GC)
  maximum time:     9.143 ms (0.00% GC)
  --------------
  samples:          863
  evals/sample:     1

In [36]:
d["Julia (hand-written, simd)"] = minimum(b.times) / 1e6

4.615583

## Summary

In [37]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Python (numpy)......................4.01
Julia (built-in)....................4.48
C (fastmath)........................4.49
Julia (hand-written, simd)..........4.62
C..................................12.83
Julia (hand-written)...............13.33
Python (built-in)..................62.56
Julia (built-in, Any).............268.39
Python (hand-written).............274.25


And of course, our hand-written Julia implementation is type-generic!

# Supplement: What about other functions?

### Log

In [38]:
@which log(1.2)

log(x::Float64) in Base.Math at special/log.jl:253

In [39]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log($(Ref(1.2))[])    
@btime clog($(Ref(1.2))[])    
@btime llvmlog($(Ref(1.2))[]);

  5.348 ns (0 allocations: 0 bytes)
  4.559 ns (0 allocations: 0 bytes)
  4.382 ns (0 allocations: 0 bytes)


On my Windows 10 machine:

```
  6.199 ns (0 allocations: 0 bytes)
  9.408 ns (0 allocations: 0 bytes)
  8.599 ns (0 allocations: 0 bytes)
```

### Exp

In [40]:
@which exp(1.2)

exp(x::T) where T<:Union{Float32, Float64} in Base.Math at special/exp.jl:74

In [41]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp($(Ref(1.2))[])    
@btime cexp($(Ref(1.2))[])    
@btime llvmexp($(Ref(1.2))[]);

  7.293 ns (0 allocations: 0 bytes)
  4.906 ns (0 allocations: 0 bytes)
  4.529 ns (0 allocations: 0 bytes)


On my Windows 10 machine:

```
  7.899 ns (0 allocations: 0 bytes)
  22.870 ns (0 allocations: 0 bytes)
  22.868 ns (0 allocations: 0 bytes)
```

### Matrix multiplication

In [1]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [2]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [3]:
C ≈ A * B

true

In [4]:
using BenchmarkTools

function mul_naive!(𝐂, 𝐀, 𝐁)
   @inbounds for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

mul_naive! (generic function with 1 method)

In [5]:
mul_naive!(C,A,B)
C ≈ A * B

true

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [6]:
using LoopVectorization

function mul_avx!(𝐂, 𝐀, 𝐁)
   @avx for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

mul_avx! (generic function with 1 method)

In [7]:
mul_avx!(C,A,B)
C ≈ A * B

true

In [8]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [9]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib_gemm = tempname()
using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib_gemm * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

c_gemm (generic function with 1 method)

In [10]:
c_gemm(C, A, B)
C ≈ A*B

true

In [12]:
@btime mul!($C, $A, $B);
@btime mul_naive!($C,$A,$B);
@btime mul_avx!($C,$A,$B);
@btime c_gemm($C, $A, $B)

  264.835 ns (0 allocations: 0 bytes)
  647.370 ns (0 allocations: 0 bytes)
  80.179 ns (0 allocations: 0 bytes)
  420.678 ns (0 allocations: 0 bytes)


**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` is slower than `mul!`.

In [ ]:
N = 100
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [ ]:
@btime mul_avx!($C, $A, $B);
@btime mul!($C, $A, $B);

However, [Octavian.jl](https://github.com/JuliaLinearAlgebra/Octavian.jl) is a (experimental) package that adds multithreading on top of LoopVectorization.jl. Again, we can blast OpenBLAS out of the water with pure Julia code. (Restart the kernel before executing the following.)

In [ ]:
Threads.nthreads()

In [ ]:
using Pkg
pkg"activate --temp"
pkg"add Octavian"
pkg"add BenchmarkTools"

In [ ]:
using Octavian, LinearAlgebra, BenchmarkTools

In [ ]:
N = 100; C = zeros(N,N); A = rand(N,N); B = rand(N,N);

In [ ]:
@btime Octavian.matmul!($C, $A, $B);

In [ ]:
@btime mul!($C, $A, $B);

**Resources/more info:**

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Performance-variation.ipynb

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb


**More comprehensive benchmarks:** https://chriselrod.github.io/LoopVectorization.jl/stable/examples/matrix_multiplication/#Matrix-Multiplication